In [1]:
import pandas as pd
import numpy as np
import gc

# Only read up to 4096 bytes, > 4096 has 100% malware rate
MAX_SIZE = 4096
TOTAL_ROWS =  113636
USE_COLS = list(range(2, MAX_SIZE))
ROWS = TOTAL_ROWS

train = pd.read_csv("./data/train.zip", nrows=ROWS, usecols=USE_COLS, header=None, names = list(range(0, MAX_SIZE)), error_bad_lines=False)
train_label = pd.read_csv("./data/train_label.csv", usecols=[1], nrows=ROWS)

train = np.nan_to_num(train, copy=False)
assert train.shape[0] == train_label.shape[0], "Train and label shapes are different"

In [2]:
mask = np.random.rand(len(train)) < 0.8

train_label = train_label.values

x_train = train[mask]
y_train = train_label[mask]
x_test = train[~mask]
y_test = train_label[~mask]

train = None
train_label = None
gc.collect()

14

In [3]:
import keras
from keras.layers import *
from keras import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras import optimizers
from keras import regularizers

own_embedding_vocab_size = 256

maxlen = 4094

model = Sequential()
model.add(Embedding(input_dim=own_embedding_vocab_size, # 10
                    output_dim=32, 
                    input_length=maxlen))
model.add(Dropout(rate=0.2))

model.add(Flatten())

model.add(Dense(256, activation='elu', activity_regularizer=regularizers.l1_l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(128, activation='elu', activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(64, activation='elu', activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(32, activation='elu', activity_regularizer=regularizers.l2(0.0001)))
model.add(Dropout(rate=0.5))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid', activity_regularizer=regularizers.l2(0.0001)))

adam=optimizers.Adam(lr=0.001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['acc'])  # Compile the model
print(model.summary())  # Summarize the model
model.fit(x_train, y_train, epochs=21, verbose=1)  # Fit the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)  # Evaluate the model
model.save('mlp_model.h5')
print('Accuracy: %0.3f' % accuracy)

/home/kai/.local/share/virtualenvs/CS5242-xoh8m7DM/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4094, 32)          8192      
_________________________________________________________________
dropout_1 (Dropout)          (None, 4094, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 131008)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33538304  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
__________

In [17]:
TOTAL_ROWS = 133223
ROWS = TOTAL_ROWS

def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                yield np.nan_to_num(line[2:MAX_SIZE], copy=False)
# test = pd.read_csv("./data/test.csv", nrows=ROWS, usecols=USE_COLS, header=None, names = list(range(0, MAX_SIZE)))
# test = np.nan_to_num(test, copy=False)
ypred = model.predict_generator(generate_arrays_from_file("./data/test.csv"), workers=1, use_multiprocessing=False, verbose=1)

df = pd.DataFrame({'sample_id': range(1, len(test) + 1), 'malware': ypred.flatten()}, columns=['sample_id', 'malware'])
df.to_csv('./data/predict.csv', index=False)

ValueError: `steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps` or use the `keras.utils.Sequence` class.